In [1]:
from torchvision.datasets import MNIST 
from torchvision import transforms 
from torch.utils.data import DataLoader
import torch 
import torch.nn.functional as F 
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [2]:
class Flatten(torch.nn.Module):
  def forward(self,img):
    img_shape = img.shape
    return img.view(img_shape[0],img_shape[1]*img_shape[2])

In [3]:
mnist_tnfs = transforms.Compose([transforms.ToTensor(),Flatten()])


In [4]:
train,test = MNIST('.', train=True, download=True,transform=mnist_tnfs), MNIST('.', train=False, download=True,transform=mnist_tnfs)


0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


0it [00:00, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
bs = 128 


In [37]:


train_dl = DataLoader(train,bs)
valid_dl = DataLoader(test,bs,shuffle=False)

In [38]:
xb,yb = next(iter(valid_dl))

In [39]:
xb,yb = next(iter(valid_dl))
assert xb.shape==(bs,1,28*28)
assert yb.shape==(bs,)

In [40]:
len(train.classes)


10

In [41]:
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()


In [42]:
n,m = train.data.shape[0],train.data.shape[1]*train.data.shape[2]
c = len(train.classes)
nh = 10

lr = 0.01   # learning rate
epochs = 30 # how many epochs to train for
n,m,c,lr,epochs

(60000, 784, 10, 0.01, 30)

In [43]:
class MNISTNN(torch.nn.Module):
  def __init__(self,m,nh,c):
    super(MNISTNN, self).__init__()
    self.lin1 = torch.nn.Linear(m,nh)
    self.act1 = torch.nn.ReLU()
    self.out = torch.nn.Linear(nh,10)
  
  def forward(self,xb):
    xb = xb.squeeze(1)
    return self.out(self.act1(self.lin1(xb)))

In [44]:
def get_model(nh):
    model = MNISTNN(m,nh,c)
    return model, torch.optim.SGD(model.parameters(), lr=lr)

In [45]:
model,opt = get_model(50)


In [46]:
loss_func = F.cross_entropy


In [47]:
loss_func(model(xb), yb)


tensor(2.2940, grad_fn=<NllLossBackward>)

In [48]:
model_layers = ['lin1','out']


In [49]:
model

MNISTNN(
  (lin1): Linear(in_features=784, out_features=50, bias=True)
  (act1): ReLU()
  (out): Linear(in_features=50, out_features=10, bias=True)
)

In [50]:

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [53]:
#export
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs),
            DataLoader(valid_ds, batch_size=bs*2))

In [54]:
train_dl,valid_dl = get_dls(train, test, bs)
model,opt = get_model(50)
loss,acc = fit(5, model, loss_func, opt, train_dl, valid_dl)

0 tensor(1.1148) tensor(0.7837)
1 tensor(0.6117) tensor(0.8615)
2 tensor(0.4707) tensor(0.8847)
3 tensor(0.4082) tensor(0.8948)
4 tensor(0.3734) tensor(0.9004)


In [32]:

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            with torch.no_grad():
                for layer_name in model_layers:
                    l = getattr(model,layer_name)
                    l.weight -= l.weight.grad * lr
                    l.bias   -= l.bias.grad   * lr
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()



        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [33]:
train_dl,valid_dl = get_dls(train, test, bs)
model,opt = get_model(50)
loss,acc = fit(5, model, loss_func, opt, train_dl, valid_dl)

0 tensor(1.0892) tensor(0.8046)
1 tensor(0.5924) tensor(0.8660)
2 tensor(0.4600) tensor(0.8870)
3 tensor(0.4024) tensor(0.8953)
4 tensor(0.3699) tensor(0.9012)


In [36]:
train[0:5]

ValueError: only one element tensors can be converted to Python scalars